In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [30]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
sys.path.append("/Users/Codes/Bitcoin_trader")

In [31]:
from positions import Positions
from order_class import BuyOrder, SellOrder

In [32]:
# stop_loss = 0.2%
# take_profit = 0.6%
np.random.seed(42)
price = np.random.uniform(low=100, high=200, size=100) 
signals = np.random.choice([-1, 0, 1], size=100)

In [33]:
positions = Positions()
for i in range(100):
    positions.check_positions(price[i])
    if signals[i] == 1:
        positions.add_position(BuyOrder(price[i], 1, stoploss=0.002, takeprofit=0.006))
    elif signals[i] == -1:
        positions.add_position(SellOrder(price[i], 1, stoploss=0.002, takeprofit=0.006))
    print(positions.get_positions())

[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>, <order_class.SellOrder object at 0x14796fe80>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>, <order_class.SellOrder object at 0x14796f3d0>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>, <order_class.SellOrder object at 0x1478ec850>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>, <order_class.BuyOrder object at 0x1478ec8e0>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder object at 0x14796f4c0>, <order_class.BuyOrder object at 0x1478ec8e0>, <order_class.BuyOrder object at 0x1478ec7f0>]
[<order_class.BuyOrder object at 0x14796f940>, <order_class.BuyOrder o

In [34]:
print(positions.profit_loss)

21.416916754006706
